In [ ]:
import configparser
config = configparser.ConfigParser()
config.read('../config/config.ini')

In [ ]:
import sys
sys.path.insert(0, '../source/')
from preprocessing_images import PreprocessingImages

In [ ]:
import pandas as pd
cols = ['Label', 'Latin Name', 'Common Name','Train Images', 'Validation Images']
label_df = pd.read_csv(f"../input/archive/monkey_labels.txt", names=cols, skiprows=1)
label_df

In [ ]:
train_generator, validation_generator = PreprocessingImages(config).preprocessImages()

In [ ]:
from cnn_model import CNNModel
model = CNNModel(config, train_generator[0][0][0]).build_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
# Create the callbacks
from keras.callbacks import TensorBoard, EarlyStopping
callbacks = [
            TensorBoard(log_dir=config['CALLBACKS']['TENSORBOARD_LOGDIR'] + "3",
                        write_images=config['CALLBACKS']['TENSORBOARD_WRITEIMAGES'],
                        write_graph=config['CALLBACKS']['TENSORBOARD_WRITEGRAPH'],
                        update_freq=config['CALLBACKS']['TENSORBOARD_UPDATEFREQ']
                        ),
            EarlyStopping(monitor=config['CALLBACKS']['EARLYSTOPPING_MONITOR'],
                          mode=config['CALLBACKS']['EARLYSTOPPING_MODE'],
                          patience=int(config['CALLBACKS']['EARLYSTOPPING_PATIENCE']),
                          verbose=1)
]

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs = int(config['CNN_CONFIGURATION']['EPOCHS']),
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=callbacks,
    verbose = 1
)

In [ ]:
from aux_functions import pltResults
# Plot accuracy graph
pltResults(
    '../logs/',
    history.history['acc'],
    history.history['val_acc'],
    'Model accuracy',
    'epoch',
    'accuracy')

In [ ]:
# Plot loss graph
pltResults(
    '../logs/',
    history.history['loss'],
    history.history['val_loss'],
    'Model loss',
    'epoch',
    'loss')